# Importing Libraries and Data

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt

In [12]:
spray = pd.read_csv('./west_nile/input/spray.csv')
weather = pd.read_csv('./west_nile/input/weather.csv')
train = pd.read_csv('./west_nile/input/train.csv')
yearly_weather = pd.read_csv('./west_nile/input/yearly_weather.csv')
test = pd.read_csv('./west_nile/input/test.csv')

# Preparing the datasets for feature engineering
### Turning the dates into datetime format

In [13]:
weather['Date'] = pd.to_datetime(weather['Date'])
train['Date'] = pd.to_datetime(train['Date'])
spray['Date'] = pd.to_datetime(spray['Date'])
test['Date'] = pd.to_datetime(test['Date'])

### Updating the Mosquito Numbers to Fix the Pipiens/Restuans Problem
#### Finding the proportions of pipiens/restuans that carry wnv and that are caught

In [14]:
# Getting a sense of the wnv proportions
train[train['WnvPresent']==1]['Species'].value_counts()

CULEX PIPIENS/RESTUANS    262
CULEX PIPIENS             240
CULEX RESTUANS             49
Name: Species, dtype: int64

In [15]:
# Determining the proportions of wnv in the two species
restuans_wnv = train[(train['WnvPresent']==1) & (train['Species']=='CULEX RESTUANS')]['WnvPresent'].sum()
pipiens_wnv = train[(train['WnvPresent']==1) & (train['Species']=='CULEX PIPIENS')]['WnvPresent'].sum()
restuans_wnv_perc = restuans_wnv/(pipiens_wnv + restuans_wnv)
pipiens_wnv_perc = 1 - restuans_wnv_perc
restuans_wnv_perc, pipiens_wnv_perc

(0.1695501730103806, 0.8304498269896194)

In [16]:
# Getting a sense of the species proportions
train['Species'].value_counts()

CULEX PIPIENS/RESTUANS    4752
CULEX RESTUANS            2740
CULEX PIPIENS             2699
CULEX TERRITANS            222
CULEX SALINARIUS            86
CULEX TARSALIS               6
CULEX ERRATICUS              1
Name: Species, dtype: int64

In [17]:
# Determining the proportions of mosquitos in the two species
restuans_mosq = train[train['Species']=='CULEX RESTUANS']['NumMosquitos'].sum()
pipiens_mosq = train[train['Species']=='CULEX PIPIENS']['NumMosquitos'].sum()
restuans_mosq_perc = restuans_mosq/(pipiens_mosq + restuans_mosq)
pipiens_mosq_perc = 1 - restuans_mosq_perc
restuans_mosq_perc, pipiens_mosq_perc

(0.34405744324689436, 0.6559425567531056)

#### Creating dummy columns for the vectors (wnv presence and musquito count)

In [18]:
# Creating columns for the vectors
train = pd.get_dummies(train, columns=['Species'])

In [19]:
# Creating columns for the number each vector
train['Num_Pipiens'] = train['Species_CULEX PIPIENS'] * train['NumMosquitos']
train['Num_Pipiens/Restuans'] = train['Species_CULEX PIPIENS/RESTUANS'] * train['NumMosquitos']
train['Num_Restuans'] = train['Species_CULEX RESTUANS'] * train['NumMosquitos']

In [20]:
# Creating columns for the wnv prevelance of each vector
train['WNV_Pipiens'] = train['Species_CULEX PIPIENS'] * train['WnvPresent']
train['WNV_Pipiens/Restuans'] = train['Species_CULEX PIPIENS/RESTUANS'] * train['WnvPresent']
train['WNV_Restuans'] = train['Species_CULEX RESTUANS'] * train['WnvPresent']

In [21]:
# Splitting up the restuans/pipiens to the restuans and pipiens columns
train['Num_Restuans'] = train['Num_Pipiens/Restuans'] * restuans_mosq_perc + train['Num_Restuans']
train['Num_Pipiens'] = train['Num_Pipiens/Restuans'] * pipiens_mosq_perc + train['Num_Pipiens']
train['WNV_Restuans'] = train['WNV_Pipiens/Restuans'] * restuans_wnv_perc + train['WNV_Restuans']
train['WNV_Pipiens'] = train['WNV_Pipiens/Restuans'] * pipiens_wnv_perc + train['WNV_Pipiens']

In [22]:
# Dropping the columns we don't need anymore
columns_to_drop = ['WNV_Pipiens/Restuans', 'Num_Pipiens/Restuans', 'Species_CULEX PIPIENS', 'Species_CULEX RESTUANS',
                  'Species_CULEX PIPIENS/RESTUANS']
train.drop(columns_to_drop, axis=1, inplace=True)

### Combining the weather data to have one entry per day
#### Dealing with the non-numeric data

In [23]:
# Determining string values and which features have them
string_dict = defaultdict(list)
for row in weather.index:
    for column in weather.columns:
        value = weather.loc[row, column]
        if type(value) == str:
            try:
                float(value)
            except:
                if value not in string_dict[column]:
                    string_dict[column].append(value)
for key, value in string_dict.items():
    print(key, value)

CodeSum [' ', 'BR', 'BR HZ', 'HZ', 'RA', 'RA BR', 'TSRA RA BR', 'RA VCTS', 'TSRA RA', 'RA HZ', 'TSRA RA BR HZ', 'TSRA BR HZ', 'RA BR HZ VCTS', 'TSRA RA HZ', 'TSRA BR HZ VCTS', 'TSRA', 'TSRA BR HZ FU', 'TSRA RA HZ FU', 'BR HZ FU', 'TSRA RA VCTS', 'HZ VCTS', 'TSRA HZ', 'VCTS', 'RA BR VCTS', 'TSRA RA BR VCTS', 'TS TSRA RA BR HZ VCTS', 'DZ BR', 'TS TSRA RA BR HZ', 'TS TSRA BR HZ', 'RA BR HZ', 'TSRA RA DZ BR HZ', 'TS TSRA RA BR', 'TS RA BR', 'TS TSRA RA', 'TS TSRA RA BR VCTS', 'TS TSRA BR', 'TS RA', 'RA BCFG BR', 'TSRA BR', 'RA DZ FG+ BCFG BR', 'RA FG+ MIFG BR', 'RA DZ', 'RA DZ BR', 'TS TSRA RA HZ', 'TSRA RA FG+ FG BR', 'TSRA DZ FG+ FG BR HZ', 'TS BR', 'RA BR SQ', 'TS TSRA', 'TSRA RA BR HZ VCTS', 'BR VCTS', 'TS', 'FG+ BR HZ', 'RA SN', 'TSRA RA DZ BR', 'DZ BR HZ', 'RA BR FU', 'TS BR HZ', 'DZ', 'FG+ BR', 'FG+ FG BR', 'FG+ MIFG BR', 'TSRA RA FG BR', 'TSRA FG+ BR', 'RA DZ BR HZ', 'RA DZ SN', 'FG+ FG BR HZ', 'TS TSRA RA FG BR', 'BR HZ VCFG', 'TS RA FG+ FG BR', 'TSRA RA FG+ BR', 'RA DZ FG+ FG BR'

In [24]:
# Finding columns with missing data
missing_dict = defaultdict(int)
for row in weather.index:
     for column in weather.columns:
        if weather.loc[row, column] in ['M', '-']:
            missing_dict[column] += 1
missing_dict

defaultdict(int,
            {'Water1': 2944,
             'Depart': 1472,
             'Sunrise': 1472,
             'Sunset': 1472,
             'Depth': 1472,
             'SnowFall': 1472,
             'Tavg': 11,
             'Heat': 11,
             'Cool': 11,
             'StnPressure': 4,
             'SeaLevel': 9,
             'AvgSpeed': 3,
             'PrecipTotal': 2,
             'WetBulb': 4})

In [25]:
# Fill in the other station's value if the value is missing
for column in missing_dict.keys():
    for row, date in weather[weather[column].isin(['M', '-'])]['Date'].items():
        if weather.loc[row, 'Station'] == 1:
            new_station = 2
        else:
            new_station = 1
        new_row = weather[(weather['Date']==date)&(weather['Station']==new_station)].index[0]
        weather.loc[row, column] = weather.loc[new_row, column]

In [26]:
# Determine where there might still be missing values
missing_dict = defaultdict(int)
for row in weather.index:
     for column in weather.columns:
        if weather.loc[row, column] in ['M', '-']:
            missing_dict[column] += 1
missing_dict

defaultdict(int, {'Water1': 2944, 'StnPressure': 2})

In [27]:
# Looking at the missing pressure data to determine what to do
weather[weather['StnPressure']=='M']

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
2410,1,2013-08-10,81,64,73,0,57,63,0,8,...,,0,M,0.0,0.00,M,30.08,5.3,5,6.5
2411,2,2013-08-10,81,68,75,0,55,63,0,10,...,,0,M,0.0,0.00,M,30.07,6.0,6,7.4


In [28]:
# Filling in the missing pressure with the average pressure for that month
aug_2013 = pd.DatetimeIndex(start='2013-08-01', end='2013-08-31', freq='d')
mean_pressure = pd.to_numeric(weather[(weather['Date'].isin(aug_2013))&(weather['Date']!='2013-8-10')]
                              ['StnPressure']).mean()
weather['StnPressure'] = weather['StnPressure'].map(lambda x: round(mean_pressure, 2) if x == 'M' else x)

In [29]:
# Dropping the column where all of the data is missing
weather.drop('Water1', axis=1, inplace=True)

In [30]:
# Filling in 0s for 'trace' values
for column in ['SnowFall', 'PrecipTotal']:
    weather[column] = weather[column].map(lambda x: 0.0 if x == '  T' else x)

In [31]:
# Converting all of the object columns (except 'CodeSum') to floats
for column, dtype in weather.dtypes.items():
    if (dtype == 'object') & (column != 'CodeSum'):
        weather[column] = pd.to_numeric(weather[column])

#### Grouping by day

In [32]:
# Grouping all of the weather data by day
weather.groupby(pd.Grouper(key='Date', freq='1d')).mean().head()

,Station,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,Depth,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
Date,,,,,,,,,,,,,,,,,,,
2007-05-01,1.5,83.5,51.0,67.5,14.0,51.0,56.5,0.0,2.5,448.0,1849.0,0.0,0.0,0.0,29.140,29.820,2.20,26.0,9.40
2007-05-02,1.5,59.5,42.5,51.5,-3.0,42.0,47.0,13.5,0.0,447.0,1850.0,0.0,0.0,0.0,29.410,30.085,13.15,3.0,13.40
2007-05-03,1.5,66.5,47.0,57.0,2.0,40.0,49.0,8.0,0.0,446.0,1851.0,0.0,0.0,0.0,29.425,30.120,12.30,6.5,12.55
2007-05-04,1.5,72.0,50.0,58.0,4.0,41.5,50.0,7.0,0.0,444.0,1852.0,0.0,0.0,0.0,29.335,30.045,10.25,7.5,10.60
2007-05-05,1.5,66.0,53.5,60.0,5.0,38.5,49.5,5.0,0.0,443.0,1853.0,0.0,0.0,0.0,29.430,30.095,11.45,7.0,11.75


In [33]:
# Redefining weather and dropping the extra days created by the grouper with all NaN values
weather = weather.groupby(pd.Grouper(key='Date', freq='1d')).mean().dropna()

### Fixing the Duplicate Traps Per Day Problem

In [38]:
# Adding the mosquito data for each trap on each day
# Checked to make sure that adding long, lat, AA, & block did not create multiples of a trap on a day for train
train['Trap'] = train['Trap'].map(lambda x: x[:4])
train = train.groupby(['Date', 'Trap', 'Longitude', 'Latitude', 'AddressAccuracy', 'Block'], as_index=False).sum()
test['Trap'] = test['Trap'].map(lambda x: x[:4])
test = test.groupby(['Date', 'Trap', 'Longitude', 'Latitude', 'AddressAccuracy', 'Block'], as_index=False).sum()

### Cleaning the yearly weather data

In [41]:
yearly_weather = yearly_weather.groupby('DATE').mean()

## Exporting the data frames

In [42]:
# These are commented out so that we don't accidentally run them.
# weather.to_csv('./weather_data')
# train.to_csv('./mosquito_data')
# yearly_weather.to_csv('./yearly_weather_data')
# test.to_csv('./test_data')